In [514]:
import pandas as pd
import numpy as np
import  requests
from collections import Counter
import re
import json

In [515]:
def readalldata(filename,lookup,read_count):
    lookup = lookup
    cnt = read_count
    found = 0
    dic = []
    myFile = open(filename)
    for num, line in enumerate(myFile):
            if lookup in line:
                print ('found at line:', num)
                print (line)
                found = num;
                cnt=1;

            if cnt<=read_count and num > found:
                dic.append(line)

                print(line);

                cnt+=1;

    myFile.close()
    return(dic)



In [516]:
dic = readalldata('TC1-setuplog.txt','show version',14)
dic1 = readalldata('sr-zap-input-file-pod.json','Topology',30)
dic2 = readalldata("sr-zap-input-file-pod.json",'feature_configuration',5083)
dic3= readalldata("xunit.json",'testSuites',16)

-Info-----2019-03-11T05-00-12[MainThread][cafy]> Analyzer initialized

found at line: 249
-Info-----2019-03-11T05-00-22[MainThread][f3a-r1-pod11:vty]> executing on RP/0/RP0/CPU0:f3a-r1-pod11.. show version

-Info-----2019-03-11T05-00-23[MainThread][Timer]> elapsed time: 0.60 s

-Out------2019-03-11T05-00-23[MainThread][f3a-r1-pod11:vty]> 

Mon Mar 11 04:00:23.021 UTC

Cisco IOS XR Software, Version 6.5.3.05I

Copyright (c) 2013-2019 by Cisco Systems, Inc.

Build Information:

 Built By     : ahoang

 Built On     : Sun Mar  3 00:58:45 PST 2019

 Built Host   : iox-ucs-027

 Workspace    : /auto/iox-ucs-027-san2/prod/6.5.3.05I.SIT_IMAGE/ncs5500/ws

 Version      : 6.5.3.05I

 Location     : /opt/cisco/XR/packages/

cisco NCS-5500 () processor

System uptime is 43 minutes

found at line: 307
-Info-----2019-03-11T05-00-34[MainThread][f3a-r3-pod11:vty]> executing on RP/0/RP0/CPU0:f3a-r3-pod11.. show version

-Info-----2019-03-11T05-00-35[MainThread][Timer]> elapsed time: 0.60 s

-Out------

              "redistribute_connected": true,

              "distribute_link_state": true,

              "nsr": true,

              "nsf_type": "cisco",

              "maximum_paths": "32",

              "srgb": {

                "srgb_lower_bound": "21000",

                "srgb_upper_bound": "30000"

              },

              "sr": {

                "sr_enable": "mpls",

                "sr_prefer": true,

                "prefix_sid_map_adv_local": true

              },

              "enable_mpls": true,

              "area": {

                "0": {

                  "peering": {

                    "routers": [

                      {

                        "interface_list": [

                          "Loopback0"

                        ],

                        "network_type": "point-to-point",

                        "prefix_sid": [

                          {

                            "prefix_sid_type": "index",

                            "pre


          "103.2.1.2": "131.0.2.1",

          "103.3.1.2": "131.0.3.1",

          "103.4.1.2": "131.0.4.1",

          "103.5.1.2": "131.0.5.1",

          "103.6.1.2": "131.0.6.1"

        }

      },

      "test_sr_ldp_tilfa": {

        "stream": [

          "SR-v4-IGP-R1-to-all",

          "BGP-v4-Global-R1->R2"

        ],

        "6pe_l3vpn_streams": [

          "BGP-6PE-R1->R2",

          "L3VPN-v4-vrf1-vrf100-R1->R2",

          "L3VPN-v6-vrf1-vrf100-R1->R2"

        ]

      },

      "test_sr_over_ldp_island": {

        "prefix_dict": {

          "100.5.1.1": {

            "mask": 32,

            "sid_start": 500,

            "sid_range": 1

          },

          "100.6.1.1": {

            "mask": 32,

            "sid_start": 600,

            "sid_range": 1

          }

        }

      },

      "test_ldp_over_sr_island": {

        "prefix_dict": {

          "100.1.1.1": {

            "mask": 32,

            "sid_start": 100,

            "sid_range":

In [517]:
columns = ['Cafe_no','TestSuit_Name','Device_Platform', 'Version','Router_count','Protocol_used','Processor/Product','Pass_count','Fail_count']
df = pd.DataFrame(columns=columns)


In [518]:
#def filtering():
#for i in range(2)
   # listvalues = []
r = re.compile(".*Software")
df['Device_Platform'] =list(set(filter(r.match, dic)))
# listvalues.append

r = re.compile(".*() processor\n")
df['Processor/Product'] = list(set(filter(r.match, dic)))


r = re.compile("(.*title.*):(.*)")
df['TestSuit_Name'] = list(filter(r.match, dic3))

r = re.compile("(.*start.*):(.*)")
df['Start_time'] = list(filter(r.match, dic3))

r = re.compile("(.*stop.*):(.*)")
df['Stop_time'] = list(filter(r.match, dic3))

r = re.compile("(.*total.*):(.*)")
df['Total_TC'] = list(filter(r.match, dic3))

r = re.compile("(.*passed.*):(.*)")
df['Pass_count'] = list(filter(r.match, dic3))

r = re.compile("(.*failed.*):(.*)")
df['Fail_count'] = list(filter(r.match, dic3))

r = re.compile("(.*duration.*):(.*)")
df['Duration'] = list(filter(r.match, dic3))


df['Version']= df['Device_Platform'].apply(lambda x: x.split(',')[1])


df['Device_Platform']= df['Device_Platform'].apply(lambda x: x.split(',')[0])

df['Processor/Product'] = df['Processor/Product'].apply(lambda x: x.split('()')[0])

df['Start_time']= df['Start_time'].apply(lambda x: x.split(':')[1])
df['Start_time']= df['Start_time'].apply(lambda x: x.split(',')[0])

df['Stop_time']= df['Stop_time'].apply(lambda x: x.split(':')[1])
df['Stop_time']= df['Stop_time'].apply(lambda x: x.split(',')[0])

df['Duration']= df['Duration'].apply(lambda x: x.split(':')[1])
df['Duration']= df['Duration'].apply(lambda x: x.split('\n')[0])

df['TestSuit_Name']= df['TestSuit_Name'].apply(lambda x: x.split(':')[1])
df['TestSuit_Name']= df['TestSuit_Name'].apply(lambda x: x.split(',')[0])

df['Total_TC']= df['Total_TC'].apply(lambda x: x.split(':')[1])
df['Total_TC']= df['Total_TC'].apply(lambda x: x.split(',')[0])

df['Pass_count']= df['Pass_count'].apply(lambda x: x.split(':')[1])
df['Pass_count']= df['Pass_count'].apply(lambda x: x.split(',')[0])

df['Fail_count']= df['Fail_count'].apply(lambda x: x.split(':')[1])
df['Fail_count']= df['Fail_count'].apply(lambda x: x.split(',')[0])

#df['Cafe_no']= 3
df.loc[:,'Cafe_no'] = 3
#df.loc[:,'TestSuit_Name']= 1

#dic1

In [422]:
#def filtering():
listvalues = {}
r = re.compile(".*Software")
listvalues['Device_Platform'] =set(filter(r.match, dic))


r = re.compile(".*() processor\n")
listvalues['Processor/Product'] = set(filter(r.match, dic))
print(listvalues)

#     r = re.compile("(.*title.*):(.*)")
#     df['TestSuit_Name'] = list(filter(r.match, dic3))

#     r = re.compile("(.*start.*):(.*)")
#     df['Start_time'] = list(filter(r.match, dic3))

#     r = re.compile("(.*stop.*):(.*)")
#     df['Stop_time'] = list(filter(r.match, dic3))

#     r = re.compile("(.*total.*):(.*)")
#     df['Total_TC'] = list(filter(r.match, dic3))

#     r = re.compile("(.*passed.*):(.*)")
#     df['Pass_count'] = list(filter(r.match, dic3))

#     r = re.compile("(.*failed.*):(.*)")
#     df['Fail_count'] = list(filter(r.match, dic3))

#     r = re.compile("(.*duration.*):(.*)")
#     df['Duration'] = list(filter(r.match, dic3))


#     df['Version']= df['Device_Platform'].apply(lambda x: x.split(',')[1])


#     df['Device_Platform']= df['Device_Platform'].apply(lambda x: x.split(',')[0])

#     df['Processor/Product'] = df['Processor/Product'].apply(lambda x: x.split('()')[0])

#     df['Start_time']= df['Start_time'].apply(lambda x: x.split(':')[1])
#     df['Start_time']= df['Start_time'].apply(lambda x: x.split(',')[0])

#     df['Stop_time']= df['Stop_time'].apply(lambda x: x.split(':')[1])
#     df['Stop_time']= df['Stop_time'].apply(lambda x: x.split(',')[0])

#     df['Duration']= df['Duration'].apply(lambda x: x.split(':')[1])
#     df['Duration']= df['Duration'].apply(lambda x: x.split('\n')[0])

#     df['TestSuit_Name']= df['TestSuit_Name'].apply(lambda x: x.split(':')[1])
#     df['TestSuit_Name']= df['TestSuit_Name'].apply(lambda x: x.split(',')[0])

#     df['Total_TC']= df['Total_TC'].apply(lambda x: x.split(':')[1])
#     df['Total_TC']= df['Total_TC'].apply(lambda x: x.split(',')[0])

#     df['Pass_count']= df['Pass_count'].apply(lambda x: x.split(':')[1])
#     df['Pass_count']= df['Pass_count'].apply(lambda x: x.split(',')[0])

#     df['Fail_count']= df['Fail_count'].apply(lambda x: x.split(':')[1])
#     df['Fail_count']= df['Fail_count'].apply(lambda x: x.split(',')[0])

#     #df['Cafe_no']= 3
#     df.loc[:,'Cafe_no'] = 3
#     #df.loc[:,'TestSuit_Name']= 1

#     #dic1
df = df.append(listvalues , ignore_index=True)

{'Device_Platform': {'Cisco IOS XR Software, Version 6.5.3.05I\n'}, 'Processor/Product': {'cisco NCS-5500 () processor\n'}}


In [519]:
#r = "(.*)(R[0-9])(.:\s)"
r = re.compile("(.*)(R[0-9])(.:\s)")
R_name = list(filter(r.match, dic1))

In [520]:
R_name = [i.split(':', 1)[0] for i in R_name]

In [521]:
R_name

['      "R1"',
 '      "R2"',
 '      "R3"',
 '      "R4"',
 '      "R5"',
 '      "R6"']

In [522]:
len(R_name)

6

In [523]:
str2 = (',').join(R_name)

In [524]:
df['Routers_Name'] = str2
df.loc[:,'Router_count'] = len(R_name)

In [525]:
Protocol_list = ["vrf",'isis','ospfv2','bgp','static-route','rpl','mpls_oam','mpls_label_range','mpls_te','rsvp','sr','accounting','acl']

In [526]:
#protocol_count = sum(list(lambda x:1 for x in dic2 if x in Protocol_list))

In [527]:

def protocol(Protocol_list,dic2):
    protocol_used = []
    protocol_count = 0
    for x in Protocol_list:
        r = re.compile(x)
        print (r)
        R_count =len(set(filter(r.findall, dic2))) # Read Note
        if R_count > 0:
            print (R_count)
            protocol_used.append(x)
            protocol_count = protocol_count +1
            print (protocol_count)
    return protocol_used,protocol_count

protocol_used,count = protocol(Protocol_list,dic2)    

re.compile('vrf')
21
1
re.compile('isis')
12
2
re.compile('ospfv2')
1
3
re.compile('bgp')
6
4
re.compile('static-route')
2
5
re.compile('rpl')
1
6
re.compile('mpls_oam')
1
7
re.compile('mpls_label_range')
1
8
re.compile('mpls_te')
4
9
re.compile('rsvp')
9
10
re.compile('sr')
91
11
re.compile('accounting')
3
12
re.compile('acl')
2
13


In [528]:
#for x in protocol_used:
str1 = ','.join(protocol_used)
df["Protocol_used"] = str1    

In [529]:
df["Total_Protocol_count"] = count

In [530]:
df.head()

,Cafe_no,TestSuit_Name,Device_Platform,Version,Router_count,Protocol_used,Processor/Product,Pass_count,Fail_count,Start_time,Stop_time,Total_TC,Duration,Routers_Name,Total_Protocol_count
0,3,"""Sr ap""",Cisco IOS XR Software,Version 6.5.3.05I\n,6,"vrf,isis,ospfv2,bgp,static-route,rpl,mpls_oam,...",cisco NCS-5500,198,43,1552294812129,1552466430969,271,171618840,"""R1"", ""R2"", ""R3"", ""R4"", ...",13


In [388]:
from sqlalchemy import create_engine


In [389]:
engine = create_engine('sqlite://', echo=False)
import pyodbc

In [392]:
df.to_sql('df1', con=engine, index_label='index',if_exists='replace')

In [393]:
engine.execute("SELECT * FROM df1").fetchall()

[(0, 3, ' "Sr ap"', 'Cisco IOS XR Software', ' Version 6.5.3.05I\n', 6, 'vrf,isis,ospfv2,bgp,static-route,rpl,mpls_oam,mpls_label_range,mpls_te,rsvp,sr,accounting,acl', 'cisco NCS-5500 ', ' 198', ' 43', ' 1552294812129', ' 1552466430969', ' 271', ' 171618840', '      "R1",      "R2",      "R3",      "R4",      "R5",      "R6"', 13)]

In [386]:
## Protocol used (name and count)

In [531]:
Master = pd.DataFrame()
Master = Master.append(df)

In [532]:
Master.head()

,Cafe_no,TestSuit_Name,Device_Platform,Version,Router_count,Protocol_used,Processor/Product,Pass_count,Fail_count,Start_time,Stop_time,Total_TC,Duration,Routers_Name,Total_Protocol_count
0,3,"""Sr ap""",Cisco IOS XR Software,Version 6.5.3.05I\n,6,"vrf,isis,ospfv2,bgp,static-route,rpl,mpls_oam,...",cisco NCS-5500,198,43,1552294812129,1552466430969,271,171618840,"""R1"", ""R2"", ""R3"", ""R4"", ...",13


# Read CSV

In [536]:
data = pd.read_csv('monitoring.csv')

In [537]:
data.head()

,test_req_id,run_list_id,reg_id,submitter,submit_time,start_time,end_time,run_time,estimated_time,test_req_type,...,label,build_status,topo_name,run_name,result_str,result_url,efr_number,priority,results,origin_run
0,114878,176.0,NaN,gobbalak,2019-04-26T09:13:23+00:00,NaN,NaN,NaN,128,python-requests/2.21.0,...,"[xr-dev""]""",Q,PSI-F3-NCS5500-NCS5508,GRE:GRE - IP,NaN,NaN,NaN,1,NaN,178066.0
1,114877,961.0,NaN,tashanka,2019-04-26T09:10:51+00:00,2019-04-26T09:11:32+00:00,NaN,NaN,386,python-requests/2.21.0,...,[],I,F6-NCS560-POD5,SR:SR,Console Log,http://devops-idt-lnx:8080/job/cafyap_pipeline...,382549.0,4,NaN,178695.0
2,114876,1345.0,NaN,annayyar,2019-04-26T09:07:51+00:00,2019-04-26T09:08:01+00:00,NaN,NaN,123,python-requests/2.21.0,...,"[non_nightly_build""]""",I,NCS5500-F90,NCS5500 Sanity:NCS5500 Sanity Production,Console Log,http://cafy-jk-lnx:8080/job/sanity_hw_pipeline...,382060.0,4,"NCS5500 Sanity Production,http://cafy-jk-lnx:8...",NaN
3,114875,2161.0,NaN,amankum4,2019-04-26T09:07:40+00:00,NaN,NaN,NaN,49,python-requests/2.21.0,...,[],Q,PSI-F3-N540-[398],L2:SPAN,NaN,NaN,NaN,4,NaN,NaN
4,114875,2161.0,NaN,amankum4,2019-04-26T09:07:40+00:00,NaN,NaN,NaN,29,python-requests/2.21.0,...,[],Q,PSI-F3-N540-[398],L2:ERSPAN,NaN,NaN,NaN,4,NaN,NaN


In [538]:
df2 = data

In [541]:
df2.columns

Index(['test_req_id', 'run_list_id', 'reg_id', 'submitter', 'submit_time',
       'start_time', 'end_time', 'run_time', 'estimated_time', 'test_req_type',
       'images', 'platform', 'variant', 'build_type', 'description',
       'test_run_id', 'suite_id', 'instance_id', 'bake', 'release', 'cases',
       'dynamic_input', 'debug', 'label', 'build_status', 'topo_name',
       'run_name', 'result_str', 'result_url', 'efr_number', 'priority',
       'results', 'origin_run'],
      dtype='object')

In [544]:
df2 = df2.drop(['run_list_id','reg_id','submitter' ,'submit_time', 'test_req_type', 'images','description','test_run_id','instance_id', 'bake', 'release', 'cases',
       'dynamic_input', 'debug', 'label','run_name', 'result_str', 'efr_number', 'priority',
       'results', 'origin_run'], axis=1)

In [545]:
df2.head()

,test_req_id,start_time,end_time,run_time,estimated_time,platform,variant,build_type,suite_id,build_status,topo_name,result_url
0,114878,NaN,NaN,NaN,128,NCS5500,NCS5508,other,207,Q,PSI-F3-NCS5500-NCS5508,NaN
1,114877,2019-04-26T09:11:32+00:00,NaN,NaN,386,NCS560,NCS560,other,13,I,F6-NCS560-POD5,http://devops-idt-lnx:8080/job/cafyap_pipeline...
2,114876,2019-04-26T09:08:01+00:00,NaN,NaN,123,NCS5500,NCS5508,mppb,243,I,NCS5500-F90,http://cafy-jk-lnx:8080/job/sanity_hw_pipeline...
3,114875,NaN,NaN,NaN,49,N540,N540,other,251,Q,PSI-F3-N540-[398],NaN
4,114875,NaN,NaN,NaN,29,N540,N540,other,251,Q,PSI-F3-N540-[398],NaN


In [549]:
df2 = df2.loc[df2['build_status'] == 'C']

In [550]:
df2.head()

,test_req_id,start_time,end_time,run_time,estimated_time,platform,variant,build_type,suite_id,build_status,topo_name,result_url
5,114874,2019-04-26T09:03:34+00:00,2019-04-26T09:09:57+00:00,6.0,80,N540,N540,other,79,C,F3A-N540-POD3,http://devops-idt-lnx:8080/job/cafyap_pipeline...
36,114850,2019-04-26T08:35:46+00:00,2019-04-26T08:52:34+00:00,17.0,164,Spitfire,Spitfirehw-sherman,other,11,C,F3B-SPITFIRE-FIX-POD2,http://cafy-jk-lnx:8080/job/cafyap_pipeline_SJ...
41,114845,2019-04-26T08:18:01+00:00,2019-04-26T08:53:18+00:00,35.0,58,Spitfire,Spitfiresim-distributed,cct,259,C,Sanity-SF-SIM-Distributed-TB4,http://cafy-sjc-1:8080/job/bit_pipeline/326//c...
50,114838,2019-04-26T08:02:31+00:00,2019-04-26T08:36:47+00:00,34.0,58,Spitfire,Spitfiresim-distributed,mppb,259,C,Sanity-SF-SIM-Distributed-TB3,http://cafy-sjc-1:8080/job/bit_pipeline/323//c...
52,114837,2019-04-26T08:02:17+00:00,2019-04-26T08:59:54+00:00,58.0,82,Spitfire,Spitfirehw-sherman,mppb,258,C,Sanity-SF-HW-Fixed-TB1,http://cafy-sjc-1:8080/job/bit_hw_pipeline/252...


In [551]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14330 entries, 5 to 18142
Data columns (total 12 columns):
test_req_id       14330 non-null int64
start_time        14330 non-null object
end_time          14330 non-null object
run_time          14330 non-null float64
estimated_time    14330 non-null int64
platform          14330 non-null object
variant           14330 non-null object
build_type        14330 non-null object
suite_id          14330 non-null int64
build_status      14330 non-null object
topo_name         14330 non-null object
result_url        14330 non-null object
dtypes: float64(1), int64(3), object(8)
memory usage: 1.4+ MB
